# Set-up

In [ ]:
import sys
!{sys.executable} -m pip install 'git+https://github.com/gaetanochiriaco/food-recognition'

In [5]:
# functions and modules from github repo
from food_recognition.train import training_loop
from food_recognition.dataloader import load_data

In [2]:
# other functions and modules
import torch
import zipfile
import os
import timm
from timm.optim import create_optimizer_v2
from timm.scheduler.step_lr import StepLRScheduler

In [3]:
# connection to gdrive folder to import data/checkpoints
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Data loading

In [6]:
# loading labels
if not os.path.exists("/content/Food2k_label"):
  with zipfile.ZipFile('/content/drive/MyDrive/TesiMagistrale/data/Food2k_label.zip', "r") as z:
    z.extractall("/content/")

In [4]:
# extracting data on disk
!tar -xf /content/drive/MyDrive/TesiMagistrale/data/Food2k_resize.tar -C /content

In [12]:
# parameters for training and data loading
image_size = (224,224)
batch_size = 16
load_check = True
mean_img = [0.5457954, 0.44430383, 0.34424934]
sd_img = [0.23273608, 0.24383051, 0.24237761]
num_classes = 2000
save_path = "/content/drive/MyDrive/TesiMagistrale/VIT/checkpoints"
model_name = "swin_base_finetune"
epochs = 35

In [13]:
# data loading
train_dataset, train_loader, test_dataset, test_loader = load_data(
    image_path= "/content/Food2k_resize",
    train_dir = "/content/Food2k_label/train_finetune.txt" ,
    test_dir = "/content/Food2k_label/test_finetune.txt",
    batch_size= batch_size,
    image_size=image_size, 
    mean_img = mean_img,
    sd_img = sd_img,
    rrci = True,
    h_flip = True,
    aug3 = True,
    color_jitter = True
)

# Model loading

In [ ]:
# base model architecture from huggingface
model = timm.create_model('swin_base_patch4_window7_224_in22k', pretrained=False, num_classes=num_classes)

In [9]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 88793224


In [14]:
# moving model on gpu and using a memory efficient format
model = model.to(memory_format=torch.channels_last)
model = model.cuda()

In [15]:
# setting up optimizer and scheduler
optimizer = create_optimizer_v2(model, opt="adamw",
    lr=0.00002)
scheduler = StepLRScheduler(optimizer, decay_t = 7, decay_rate=.5)

In [ ]:
# loading checkpoint containing model, optimizer and scheduler state
list_checkpoint = os.path.exists(save_path+f"/{model_name}.pt")
if load_check and list_checkpoint:
  checkpoint = torch.load(save_path+f"/{model_name}.pt")
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
  last_batch = checkpoint['batch']
else:
  last_epoch = 0 
  last_batch = 0

# Training

In [ ]:
# training model from last batch done
training_loop(
    model = model,
    loader = train_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs = 35,
    last_batch = last_batch,
    mixup = False,
    save = False,
    bert = True,
    label_path = "/content/drive/MyDrive/TesiMagistrale/data/food2k_label.csv"
)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


epoch: 19  batch:  100 [  1600/620192]  loss: 1.23376513 accuracy:  75.000% avg acc:  72.938%
epoch: 19  batch:  200 [  3200/620192]  loss: 0.72966564 accuracy:  75.000% avg acc:  73.406%
epoch: 19  batch:  300 [  4800/620192]  loss: 0.48975646 accuracy:  93.750% avg acc:  73.729%
epoch: 19  batch:  400 [  6400/620192]  loss: 0.75435507 accuracy:  62.500% avg acc:  73.953%
epoch: 19  batch:  500 [  8000/620192]  loss: 1.14126253 accuracy:  62.500% avg acc:  73.800%
epoch: 19  batch:  600 [  9600/620192]  loss: 0.84262311 accuracy:  81.250% avg acc:  73.865%
epoch: 19  batch:  700 [ 11200/620192]  loss: 1.01205134 accuracy:  75.000% avg acc:  73.848%
epoch: 19  batch:  800 [ 12800/620192]  loss: 0.65724885 accuracy:  75.000% avg acc:  73.852%
epoch: 19  batch:  900 [ 14400/620192]  loss: 1.00199664 accuracy:  62.500% avg acc:  73.785%
epoch: 19  batch: 1000 [ 16000/620192]  loss: 0.67282236 accuracy:  81.250% avg acc:  73.812%
epoch: 19  batch: 1100 [ 17600/620192]  loss: 0.95201492 acc